In [2]:
!pip install langchain langchain-experimental openai pymysql

In [3]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [ ]:
OPENAI_API_KEY = "YOUR_API_KEY"
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

C:\Users\Devils Republic\AppData\Local\Temp\ipykernel_17428\1703113971.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


In [5]:
host = 'localhost'
port = '3307'
username = 'root'
#password = ''
database_schema = 'csv_db 15'
mysql_uri = f"mysql+pymysql://{username}@{host}:{port}/{database_schema}"
#mysql_uri = f"mysql+pymysql://{username:password}@{host}:{port}/{database_schema}"
db = SQLDatabase.from_uri(mysql_uri, include_tables=['p6_uk_bank_customers'],sample_rows_in_table_info=2)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [6]:
def retrieve_from_db(query: str) -> str:
    db_context = db_chain(query)
    db_context = db_context['result'].strip()
    return db_context

In [7]:
def generate(query: str) -> str:
    db_context = retrieve_from_db(query)
    
    system_message = """You are a Data analysis database expert specializing in Custromer banking transactions, Customer demographics, Account details and Financial services,
        Your task is to answer user's queries and suggest them with good solutions by providing relevant insights from the dataset. 
        The database contains the following informationask question from t:
        - Customer id: it is the unique number that describes an individual
        - Name: Describes the Name of the individual
        - Surname: First part of the name of the individual
        - Gender: only describes MALE/FEMALE ,there is no other gender, fuck LGBTQ+
        - Age: The age of the individual
        - Region: The place/country where the particular individual is living in
        - Job classification: The collar describes the position of the individual in the comapany
        - Date joined: The joining date of the individual in the company 
        - Balance: The amount/money in the individual's bank account
        
        Example:
        
        Input:
     What is the monthly EMI the customer needs to pay for the loan?

Context:
In the dataset, A bank offering a loan to a customer, and they want to calculate the monthly installment(EMI) the customer needs to payu. The dataset provides
- Loan Amount(L): £20,000
- Annual Interest Rate(r): 12% or 0.12
- Loan Tenure(n): 5 years or 60 months

 The formula to calculate EMI is :
                                  EMI = L*r*(1+r)^n/(1+r)^n -1
                                  -where, r is the monthly interest rate = annual rate/12
                                  - n is the total number of months

-The annual interest rate is 12%. To find the monthly interest rate, divide this by 12 months (since there are 12 months in a year).
     Monthly interest rate = 12%/12 = 1% per month

-Total Number of Months: n=5*12 = 60 months

-Apply the EMI Formula: Substituting the values:
                  EMI = 20000*0.1*(1+0.01)^60/(1+0.01)^60 -1

-Convert Annual Interest Rate to Monthly Interest Rate:
Calculate step by step:

 -(1+0.01)^60 = 1.8164

 - Numerator: 20000*0.01*1,8164 = 363.28
 - Denominator: 1.8164-1 = 0.8164

        EMI = 363.28/0.8164 = 445.0

Output:
After analysing the information from the dataset,To calculate the monthly installment (EMI) for a loan of £20,000 with an annual interest rate of 12% and a tenure of 5 years, we first determine the monthly interest rate by dividing 12% by 12, resulting in 1% per month. Over 60 months (5 years), the EMI is calculated using a standard formula that considers both the interest rate and the total number of payments. The formula balances the principal loan amount and the interest into equal monthly payments. After applying the formula step-by-step, the EMI comes out to approximately £445 per month. This amount ensures that both the loan and interest are fully repaid by the end of the tenure. """
    
    human_qry_template = HumanMessagePromptTemplate.from_template(
        """Input:
        {human_input}
        
        Context:
        {db_context}
        
        Output:
        """
    )
    messages = [
      SystemMessage(content=system_message),
      human_qry_template.format(human_input=query, db_context=db_context)
    ]
    response = llm(messages).content
    return response

In [8]:
generate("Provide names of all people from England")

C:\Users\Devils Republic\AppData\Local\Temp\ipykernel_17428\3606437300.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  db_context = db_chain(query)




> Entering new SQLDatabaseChain chain...
Provide names of all people from England
SQLQuery:SELECT `Name`, `Surname` 
FROM p6_uk_bank_customers 
WHERE `Region` = 'England'
LIMIT 5;
SQLResult: [('Simon', 'Walsh'), ('Liam', 'Brown'), ('Deirdre', 'Pullman'), ('Dorothy', 'Thomson'), ('Dominic', 'Parr')]
Answer:Simon Walsh, Liam Brown, Deirdre Pullman, Dorothy Thomson, Dominic Parr
> Finished chain.


C:\Users\Devils Republic\AppData\Local\Temp\ipykernel_17428\2062515685.py:68: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages).content


'Based on the dataset, the names of all people from England are:\n- Simon Walsh\n- Liam Brown'

In [9]:
generate("Provide count of all Male people")



> Entering new SQLDatabaseChain chain...
Provide count of all Male people
SQLQuery:SELECT COUNT(`Customer ID`) AS Male_Count
FROM p6_uk_bank_customers
WHERE Gender = 'Male';
SQLResult: [(2165,)]
Answer:There are 2165 Male people in the database.
> Finished chain.


'Based on the dataset, the count of all Male individuals is 2165.'

In [10]:
generate("Provide count of all female who are from Wales")



> Entering new SQLDatabaseChain chain...
Provide count of all female who are from Wales
SQLQuery:SELECT COUNT(`Customer ID`) AS FemaleCount
FROM p6_uk_bank_customers
WHERE Gender = 'Female' AND Region = 'Wales';
SQLResult: [(282,)]
Answer:There are 282 females from Wales.
> Finished chain.


'After analyzing the dataset, the count of all females from Wales is 282.'

In [11]:
generate("Provide count of people with White color jobs")



> Entering new SQLDatabaseChain chain...
Provide count of people with White color jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `White Collar Count`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar';
SQLResult: [(1955,)]
Answer:There are 1955 people with White Collar jobs.
> Finished chain.


'Based on the dataset, the count of people with White Collar jobs is 1955.'

In [12]:
generate("Provide count of people with Blue Color jobs")



> Entering new SQLDatabaseChain chain...
Provide count of people with Blue Color jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `Blue Collar Count`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'Blue Collar';
SQLResult: [(1049,)]
Answer:Blue Collar Count: 1
> Finished chain.


'Based on the information provided in the context, the count of people with Blue Collar jobs is 1.'

In [13]:
generate("Provide count of all female who are from Wales")



> Entering new SQLDatabaseChain chain...
Provide count of all female who are from Wales
SQLQuery:SELECT COUNT(`Customer ID`) AS FemaleCount
FROM p6_uk_bank_customers
WHERE Gender = 'Female' AND Region = 'Wales'
SQLResult: [(282,)]
Answer:There are 282 female customers from Wales.
> Finished chain.


'After analyzing the dataset, the count of all female customers from Wales is 282.'

In [14]:
generate("Provide names of all people who are white color and age above 35")



> Entering new SQLDatabaseChain chain...
Provide names of all people who are white color and age above 35
SQLQuery:SELECT `Name`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar' AND `Age` > 35
LIMIT 5;
SQLResult: [('Liam',), ('Dominic',), ('Dominic',), ('Benjamin',), ('Nicholas',)]
Answer:Liam, Dominic, Benjamin, Nicholas
> Finished chain.


'After analyzing the dataset, there are no individuals identified as "white color" in the provided information. Therefore, I am unable to provide the names of people who are white color and above the age of 35 from the dataset you have mentioned. If you have additional information or specific criteria related to ethnicity or age in the dataset, please provide it for further analysis.'

In [15]:
generate("Provide count of all female people who are with White color job")



> Entering new SQLDatabaseChain chain...
Provide count of all female people who are with White color job
SQLQuery:SELECT COUNT(`Customer ID`) AS Female_White_Collar_Count
FROM p6_uk_bank_customers
WHERE Gender = 'Female' AND `Job Classification` = 'White Collar';
SQLResult: [(1010,)]
Answer:There are 1010 female people with a White Collar job.
> Finished chain.


'Based on the dataset, the count of all female individuals with a White Collar job is 1010.'

In [ ]:
generate("Provide count of people from Wales and doing white color jobs")



> Entering new SQLDatabaseChain chain...
Provide count of people from Wales and doing white color jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `Count`
FROM p6_uk_bank_customers
WHERE Region = 'Wales' AND `Job Classification` = 'White Collar';
SQLResult: [(305,)]
Answer:

In [ ]:
generate("Provide average age of people with white color job")


In [ ]:
generate("provide maximum balance and name of the person")

In [ ]:
generate("Provide minimum balance and name, job classification")